In [0]:
!pip install jieba
!pip install kashgari-tf

In [0]:
!head -n 8000 train_5_cnn.txt > comment_t.txt
!tail -n 2000 train_5_cnn.txt > comment_v_1.txt
!head -n 1500 comment_v_1.txt > comment_test.txt
!tail -n 500 comment_v_1.txt > comment_v.txt

In [0]:
import tqdm
import jieba

def read_data_file(path):
    lines = open(path, 'r', encoding='utf-8').read().splitlines()
    x_list = []
    y_list = []
    for line in tqdm.tqdm(lines):
        rows = line.split(',')
        if len(rows) >= 2:
            y_list.append(rows[1])
            x_list.append(rows[0])
        else:
            print(rows)
    return x_list, y_list

test_x, test_y = read_data_file('/content/comment_test.txt')
train_x, train_y = read_data_file('/content/comment_t.txt')
val_x, val_y = read_data_file('/content/comment_v.txt')

100%|██████████| 500/500 [00:00<00:00, 130826.70it/s]


In [0]:
print(train_x[1:2])
# print(train_y[1:4])
#print(len(train_x))

['味道\t不错\t分量\t足\t客人\t很多\t满意\t']


In [0]:
from kashgari.tasks.classification import CNN_Model
from kashgari.tasks.classification import CNN_GRU_Model
from kashgari.tasks.classification import KMax_CNN_Model
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.tasks.classification import AVCNN_Model
from kashgari.tasks.classification import AVRNN_Model
#model = CNN_GRU_Model()
model = AVRNN_Model()
model.fit(train_x, train_y, test_x, test_y, batch_size=1024,epochs = 15)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 103)]        0                                            
__________________________________________________________________________________________________
layer_embedding (Embedding)     (None, 103, 100)     176800      input[0][0]                      
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 103, 100)     0           layer_embedding[0][0]            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 103, 120)     57960       spatial_dropout1d_1[0][0]        
____________________________________________________________________________________________

In [0]:
model.evaluate(val_x, val_y)

              precision    recall  f1-score   support

           0     0.9876    0.9803    0.9840       407
           1     0.9167    0.9462    0.9312        93

    accuracy                         0.9740       500
   macro avg     0.9521    0.9633    0.9576       500
weighted avg     0.9744    0.9740    0.9742       500



In [0]:
model.save('./model')

# new_model = CNNModel.load_model('./model')
# news = """「DeepMind 击败人类职业玩家的方式与他们声称的 AI 使命，以及所声称的『正确』方式完全相反。」
# DeepMind 的人工智能 AlphaStar 一战成名，击败两名人类职业选手。掌声和欢呼之余，它也引起了一些质疑。在前天 DeepMind 举办的 AMA 中，AlphaStar 项目领导者 Oriol Vinyals 和 David Silver、职业玩家 LiquidTLO 与 LiquidMaNa 回答了一些疑问。不过困惑依然存在……
# """
# x = list(jieba.cut(news))
# new_model.predict(x)

In [0]:
import jieba 
jieba.load_userdict('/content/userdict.txt')
with open('/content/stopWords.txt') as f:
    stopWords = [line.strip() for line in f.readlines()]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.944 seconds.
DEBUG:jieba:Loading model cost 0.944 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [0]:
test_sentence = "有苍蝇。对一个才学了节肢动物的人来说，整个人都不好了 "
#classifier = CNNModel.load_model('./model')
predict_list = []
words = jieba.lcut(test_sentence)
words = [w for w in words if w not in stopWords]
print(words)
predict_list.append('\t'.join(words))
print(predict_list[0])
pred = model.predict(predict_list)
print(pred)
#print("分类为：" + str(pred))

['苍蝇', '才学', '节肢动物', '人', '整个', '人', '不好', ' ']
苍蝇	才学	节肢动物	人	整个	人	不好	 
['1']


In [0]:
def predict_category(sentence):
    wl = jieba.lcut(sentence)
    wl = [w for w in wl if w not in stopWords]
    wl = ' '.join(wl)
    rs = model.predict(wl)
    #print("predict result: {}".format(rs))
    return rs

In [0]:
import csv
import pandas as pd
#file_object2=open('/content/test_new2.csv').read()  #一行行的读取内容
test_data = pd.read_csv('/content/test_4.csv')
print(test_data.head())

                                             comment
0       糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试
1  满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...
2                              好喝！经常会再去买来喝！就是排队的人太多。
3                              三个人订的四人餐，菜量大没吃完，问道不错。
4  好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！喜欢所有菜和肉，两女一男吃两份两人餐没吃...


In [0]:
result = []
predict_list2 = []
for index, row in test_data.iterrows():
    #print(index)
    #print(row["comment"])
    words = jieba.lcut(row["comment"])
    words = [w for w in words if w not in stopWords]
    #print(words)
    predict_list2.append('\t'.join(words))
    #result.append(predict_category(row["comment"]))
#print(result)
print(predict_list2[100:103])

['牛\t蹄筋\t真不知道\t肉\t骨头\t连起\t味道\t没有\t实体店\t好吃', '一如既往\t好\t态度\t越来越\t新意', '真的\t差评\t才\t没有\t忍着\t退货\t真的\t晚\t接近\t小时\t不说\t时间\t四个\t电话\t不接\t不能\t准时\t送到\t早\t说\t \t突\t况\t打个\t电话\t要死\t \t不\t确定\t取消\t重新\t下单\t快\t苦苦\t快\t真的\t商家\t有毒\t不信\t试试\t真是\t差评\t少点\t人\t骗\t才\t确认\t收单\t恶心']


In [0]:
result = model.predict(predict_list2)

In [0]:
print(f"{result}")

['0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1',